<a href="https://colab.research.google.com/github/Shaker-emad/Shaker-emad/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from urllib import response
import nltk 
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
from tensorflow.python.framework import ops

import random
import json
import pickle



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
with open("/content/intents (4).json") as file:
    data = json.load(file)


In [5]:
train = input("Train the model type in (train): ")


Train the model type in (train): train


In [8]:
#Data preprocessing start
if train.lower() == "train":
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent['tag'])

        if intent["tag"] not in labels:
            labels.append(intent['tag'])


In [9]:
    #Getting rid of duplicate words
    words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)



In [10]:
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        #Bag of words
        bag = []

        wrds = [stemmer.stem(w) for w in doc]

        #Going through all the words
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)



In [11]:
    try:
        with open('data.pickel', 'rb') as f:
            words, labels, training, output = pickle.load(f)
    except:
        # rest of the code goes here ...
        with open('data.pickel', 'wb') as f:
            pickle.dump((words, labels, training, output), f)
    else:
        # the data has already been pickled, no need to preprocess it again
        pass



In [ ]:
#Model creation start
ops.reset_default_graph()
#Input layer trying to get the length from none on
net = tflearn.input_data(shape= [None, len(training[0])])
net = tflearn.fully_connected(net, 16, activation="relu") #hidden layer
net = tflearn.fully_connected(net, 16, activation="relu") #hidden layer
#Output layer solfmax 
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
#end model creation




In [13]:
if train.lower() == "train":
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

else:
    model.load("model.tflearn")





Training Step: 10999  | total loss: 0.00977 | time: 0.054s
| Adam | epoch: 1000 | loss: 0.00977 - acc: 1.0000 -- iter: 80/81
Training Step: 11000  | total loss: 0.00927 | time: 0.061s
| Adam | epoch: 1000 | loss: 0.00927 - acc: 1.0000 -- iter: 81/81
--


In [14]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words if word not in "?"]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)




In [ ]:
#Generate user input
def chat():
    #user input
    print("I'm the chatbot of Buffalo Burger Restaurant")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        
        #All this is going to give us a matrix of numbers where the numbers are probabilities of each class
        results = model.predict([bag_of_words(inp, words)])
        #Argmax will grab the index of highest probability in the matrix
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))


chat()

I'm the chatbot of Buffalo Burger Restaurant
You: Hi
 hello!,I'm the chatbot of Buffalo Burger Restaurant! how can I help?
You: what your phone
You can contact at: 01287414593
You: what menu
menu is : Steak ,cheeseburger, bacon burger,veggie burger,,Chicken dishes,French fries,    
